In [1]:


## CSCI E-118 Introduction to Blockchain and Bitcoin 

### Staking Chain Demo



The stakes for poker games have grown so much in the last few decades that the biggest games can only be played responsibly by billionaires.  Poker is a very technical game and the best poker pros are orders of magnitude better than these players but must be careful to avoid gambler’s ruin.  Knowing what proportion of your bankroll to risk such that your risk of ruin is always zero is determined by the Kelly Criterion.  This involves aggressively moving up or down in stakes to maximize your expectation.  It also involves being able to estimate your winrate.  To allow more granular control over these aggressive bankroll moves, selling action to investors is a very helpful tool.  This application will allow players to focus less on networking and request to play tournaments from investors that have been given access to the DApp.  

My project will be a DApp that will allow record keeping for a decentralized poker staking network.  Players in the database are able to request to play a tournament for say $10,000 and say how much of that they want to sell with an optional markup (fee) on top.  The markup is used by better players because if you have a very high ROI this is a very profitable investment in the long run.  I used Solidity (Remix) and Python (Jupyter Notebook) with Web3.py to build this DApp.  I will present the demo using explanatory text in a Jupyter Notebook that demonstrates a few sample runs on the program.  

Right now the DApp only allows players to request to play tournaments (and not cash games).  This is useful because the results are published in multiple reputable locations on the internet.  A last mile issue is needing some sort of Oracle to update the results so that the correct results for tournaments are issued to the DApp.  For now the owner of the contract is the Oracle and this is clearly a central point of failure.  All of the numbers I used were uint256 which will take more gas for some numbers that don't need to be that big.  If this app ends up being widely used it will be very inefficient to not have appropriate sized ints for numbers that won’t get too big.  Another scaling issue is currently when the player is distributing funds if one of the blocks fail then the entire function will revert and nobody will get paid.  This means players couldn’t currently try and get Bernie Sanders/Donald Trump style crowdfunding because there could be issues if the application were trying to pay out profit to thousands of addresses.  
The owner of the contract is the Oracle for results right now so one possible thing players could do is say that they played a tournament and collect the investments, but not actually play the tournament.  They would pocket the money and report the tournament as a loss.  In the future when there is an Oracle it would be able to verify that players played as well as verifying the profit results.  This would prevent errors from the owner leading to incorrect payouts.  I had to scale my project back a bit so there aren’t some very strong decentralization properties.  That being said the best decentralization property is the ability to find investors without having to go searching.  This would create two-sided marketplaces (for ex. Uber or Airbnb) where investors are able to find players who want to sell some action.  In some ways this project is a primitive cousin of the Automated Market Maker revolution that is happening in the Decentralized Finance space.  Also, all of the record keeping and transfer of funds would be done securely.  Sometimes shady investors won’t pay for a loss after the fact, or greedy players will not pay out winnings.  


Tracking funds and results in a distributed Poker Staking Network

The purpose of this application is to provide infrastructure for a distributed Poker Staking Network.  


In [2]:
from solcx import set_solc_version, compile_files
from web3 import Web3

from eth_tester import EthereumTester, PyEVMBackend

import os

In [3]:
set_solc_version('v0.7.6')

In [4]:
# EthereumTester used explicitly to get accounts
TESTER = EthereumTester(backend=PyEVMBackend())

 `get_contract_path` gets the path to the contract we want by the contract's name.

 The `compile_contract` function takes a contract name and compiles the contract with that name. It returns the compiled code for the compiled contract. The compiled code includes the Application Binary Interface (`abi`), the bytecode (`bin`), etc.

In [5]:
def get_contract_path(contract_name):
    '''
    construct path to contract file
    assumes there is a subdirectory in your current working directory named "contracts"
    
    contract_name: the name of the contract (without the .sol suffix)
    '''
    return os.path.join(os.getcwd(), 'contracts', f'{contract_name}.sol')

def compile_contract(contract_name):
    '''
    compile contract and get the result of compilation
    
    contract_name: the name of the contract, which should match the filename (without the .sol suffix)
    '''
    source_file_name = get_contract_path(contract_name)
    compiled_sol = compile_files([source_file_name]) # Compiled source code
    return compiled_sol[source_file_name + ":" + contract_name]

`get_w3` returns an instance of a web3 object, which is what we use to interact with the blockchain.

It first instantiates the web3 object with a `provider`. You can read all about it [here](https://web3py.readthedocs.io/en/stable/providers.html), but in general it's what gives us access to the blockchain. For now we'll use `EthereumTesterProvider` which integrates directly with the `eth-tester` library we're using.

The `account` paramater will be set to the default account. This is the account that we'll be conducting transactions from, unless we specify otherwise.

In [6]:
def get_w3(provider, account=None):
    '''
    get a web3 object instance
    sets the default account to the one provided or the first test account if not provided
    
    provider: backend test provider
    account:  the account number to set as default, or None if you want to use the first test account
    '''
    # web3.py instance
    w3 = Web3(provider)
    
    if account:
        # set account as default sender of transactions
        w3.eth.defaultAccount = account
    else:
        # set the first test account as default sender of transactions
        w3.eth.defaultAccount = w3.eth.accounts[0]
    
    return w3

`deploy_contract` deploys the contract to the blockchain.

`deploy_contract` takes in the `w3` object from above, which is intialized with a `provider` and default account. This is the account that the deployment will be associated with.

The second argument, `compiled_contract` is the compiled contract code, which includes the `abi` and `bin`.

`deploy_contract` also takes in a variable amount of optional args, called `c_args`. If provided, these arguments are passed into the contract's constructor. If the smart contract was written with a constructor that takes in some parameters, this is how we provide them.

Within the `deploy_contract` function, the contract interface is retrieved from the `abi` and `bin`. The interface provides the contract functions, including the contstructor to initialize it. The constructor is called, passing in any arguments and then `transact()` deploys the contract to the backend (`PyEVM`).

This provides us with a transaction hash. We use this hash as an identifier. We wait for the transaction to complete (mined and put into a block), providing the transaction hash. Once it completes, we get a receipt, which among other things contains the contract's address. We get the deployed contract instance by providing that address, along with the contract's `abi` and `bin`.

`deploy_contract` returns a tuple containg the transaction receipt--which contains information like the transaction's index, block number, gas used, contract address, etc.--and the deployed contract interface. The `deployed_contract_interface` is a `Web3` object, initialized with the compiled contract (`abi` and `bin`) as well as the address of the contract on the blockchain.

In [7]:
def deploy_contract(w3, compiled_contract, *c_args):
    '''
    deploy the contract
    
    w3: web3 object instance
    compiled_contract: dictionary of compiled contract should have values for the keys: "abi" and "bin"
    *c_args: a variable number of contract arguments to pass to the constructor of the contract
    '''
    # get the contract interface from the compiled contract
    contract_interface = w3.eth.contract(abi=compiled_contract['abi'], bytecode=compiled_contract['bin'])
    
    # Instantiate the contract by passing in constructor args, and submit the transaction to deploy
    tx_hash = contract_interface.constructor(*c_args).transact()
    
    # Wait for the transaction to be put into a block, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Get the deployed contract instance from the blockchain with the newly-deployed contract's address
    deployed_contract_interface = w3.eth.contract(
        address=tx_receipt.contractAddress,
        abi=compiled_contract['abi'],
        bytecode=compiled_contract['bin']
    )
    
    return tx_receipt, deployed_contract_interface

Accounts provided by eth-tester:

In [8]:
TESTER.get_accounts()

('0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69',
 '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718',
 '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276',
 '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141',
 '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb',
 '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C',
 '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c',
 '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528')

The staking_chain_account is the main account and this is the account that will allow backers and players into the network.  This is the default account. 
pp_bank accounts are player's addresses
backer_accounts are investor's addresses

In [9]:
staking_chain_account = TESTER.get_accounts()[0]
pp_bank1 = TESTER.get_accounts()[1]
pp_bank2 = TESTER.get_accounts()[2]
backer_account1 = TESTER.get_accounts()[3]
backer_account2 = TESTER.get_accounts()[4]
backer_account3 = TESTER.get_accounts()[5]
backer_account4 = TESTER.get_accounts()[6]

This gets the contract's compiled code, including the `abi` and `bin` (the bytecode).

In [10]:
sc_compiled = compile_contract("StakingChain")

Now we'll set up the provider. This is the connection to the blockchain.  We'll use `EthereumTesterProvider` which integrates directly with the `eth-tester` library we're using.

In [11]:
provider = Web3.EthereumTesterProvider(ethereum_tester=TESTER)

With the provider, we'll get the web3 object, passing in the default account (the staking chain account which allows backers,players into system and will report results after tournaments have finished). The returned `Web3` object, connected to the `PyEVM` backend through `eth-tester`, is our way of interfacing with the blockchain.

In [12]:
sc_w3 = get_w3(provider, staking_chain_account)

Finally, we'll deploy the staking chain contract by passing in the web3 object and the compiled code. We'll get back the receipt for the contract deoployment transaction, and the deployed contract's interface.

In [13]:
sc_receipt, sc_contract = deploy_contract(sc_w3, sc_compiled)

In [14]:
sc_contract.all_functions()

[<Function _updateResults(uint256,uint256,uint256)>,
 <Function accept_results(uint256,uint256)>,
 <Function action_owned(uint256,address,uint256)>,
 <Function backerTransPlayer(uint256,uint256)>,
 <Function distribute_result(uint256,uint256)>,
 <Function get_amount_owed(uint256,uint256)>,
 <Function invest(uint256,uint256)>,
 <Function owner()>,
 <Function play_id()>,
 <Function player_database(uint256)>,
 <Function register_pp(address,uint256,uint256)>,
 <Function request_buyin(uint256,uint256,uint256,uint256,uint256)>,
 <Function total_results(uint256)>,
 <Function tourney_results(uint256,uint256)>,
 <Function updateMaxStake(uint256,uint256)>]

`exec_call` will issue a call. This means the function is called locally, and no transaction is made on the blockchain. Therefore issuing a call does not lead to a change in the blockchain's state. Calls do not cost gas. It takes the contract interface, which is the `Web3` object initialized with the compiled contract and its address; the name of the function within the smart contract that we wish to call; and `f_args` which is a variable number of arguments we wish to pass to the smart contract function we're calling.

`exec_transact_receipt` will issue a transaction. This means the function is broadcast to the blockchain, and the blockchain's state is changed. Transactions cost gas. It returns whatever value is returned by the smart contract, as well as the transaction receipt.

`exec_transact` just issues the transaction and returns the smart contract's return value, without the receipt.

All three functions take an optional transaction dictionary. This dictionary contains values for the various fields of the transactions. Most times we can rely on the default values, but sometimes we may want to specify a different value. You can read about the transaction dictionary in the docs [here](https://web3py.readthedocs.io/en/stable/contracts.html#web3.contract.ContractFunction.transact) and [here](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.send_transaction).

In [15]:
def exec_call(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute a call, which does not execute a transaction (i.e. no write)
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    func_inst = contract_interface.get_function_by_name(function_name)

    return_value = func_inst(*f_args).call(transaction)
    return return_value

def exec_transact_receipt(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute a transaction (i.e. a write), and return the transaction receipt
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    func_inst = contract_interface.get_function_by_name(function_name)
    
    # get the return value first, without executing transaction
    return_value = exec_call(contract_interface, function_name, *f_args, transaction=transaction)
    
    # execute the transaction
    tx_hash = func_inst(*f_args).transact(transaction)
    # receipt does not contain values returned by function
    tx_receipt = contract_interface.web3.eth.waitForTransactionReceipt(tx_hash)
    
    return return_value, tx_receipt

def exec_transact(contract_interface, function_name, *f_args, transaction={}):
    '''
    execute transaction, but ignore the transaction receipt
    
    contract_interface: web3 object initialized with compiled contract and its address
    function_name: name of the function in the smart contract to be invoked
    f_args: variable number of arguments to pass in to the function in the smart contract
    transaction: dictionary containing transaction fields
    '''
    rv, _ = exec_transact_receipt(contract_interface, function_name, *f_args, transaction=transaction)
    return rv

Now we will register two poker players with max_stake set to 1000.  This is the maximum buy-in they can request to play.

In [16]:
pp_id_1, pp_id_1_receipt = exec_transact_receipt(sc_contract, "register_pp", pp_bank1, 0, 1000)
pp_id_2, pp_id_2_receipt= exec_transact_receipt(sc_contract, "register_pp", pp_bank2, 0, 1000)

print("pp_id_1: ", pp_id_1)
print("pp_id_2: ", pp_id_2)

pp_id_1:  0
pp_id_2:  1


In [17]:
pp_id_1_receipt

AttributeDict({'transactionHash': HexBytes('0xaffc06fb1d17196c6bafc43e306daefceca82caaf2d9690a9912d5d552d4553c'),
 'transactionIndex': 0,
 'blockNumber': 2,
 'blockHash': HexBytes('0x42fe24e43cb1d626bf77dedf1b4014c1e446f1dedebfe6ad5980129c6fa6d782'),
 'cumulativeGasUsed': 115867,
 'gasUsed': 115867,
 'contractAddress': None,
 'logs': [],
 'status': 1})

Player 1 requests backing for Tourney ID: 202021 Buy-in: 20 


In [18]:
_, buyin_receipt = exec_transact_receipt(sc_contract, "request_buyin", pp_id_1, 10, 20, 202021, 1000000, transaction={'from': pp_bank1})

Player 2 requests backing for Tourney ID: 502021 Buy-in: 50

In [19]:
_, buyin_receipt2 = exec_transact_receipt(sc_contract, "request_buyin", pp_id_2, 25, 50, 502021, 2000000, transaction={'from': pp_bank2})

In [20]:
exec_call(sc_contract, "play_id")

2

After the player emits an event requesting a buy-in, the contract will use an event listener so that a backer that wants to invest given these parameters can emit their own event and eventually send money.  

In [21]:
#Event listener for request_buyin()

In [22]:
from web3.logs import STRICT
#sc_contract.events.RequestTourney().processReceipt(pp_id_1_receipt, errors=STRICT)

In [23]:
def get_buyin_requested(pp_id_1_receipt):
    return sc_contract.events.RequestTourney().processReceipt(pp_id_1_receipt)[0]['args']

The details of the requested buy-in, player id, tourney_id and the markup this player is charging.  The markup is the "fee" good players will charge on top of the percentage of action * tournament buy-in.  It's possible a potential investor wants to invest in the player but if the player is too greedy, the backer will respond to the request.  

In [24]:
buyin_event1 = get_buyin_requested(buyin_receipt)
buyin_event1

AttributeDict({'amountReq': 10,
 'play_id': 0,
 'buy_in': 20,
 'tourney_id': 202021,
 'markup': 1000000})

Check the current state of player pp_id_1 after the request event was submitted.  

In [25]:
exec_call(sc_contract, "player_database", pp_id_1)

[0,
 0,
 1000,
 True,
 False,
 202021,
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 10,
 0,
 1000000,
 20]

In [26]:
#Event listener for invest()

There will be an event listener after the backer decides to invest in the player.  Backer_account1 transfers funds to pp_id_1

In [27]:
_, invest_receipt = exec_transact_receipt(sc_contract, "invest", 10, pp_id_1, transaction={'from': backer_account1, 'value': 10})
#exec_call(sc_contract, "player_database", pp_id_1)

In [28]:
sc_contract.events.PlayerNeedsFunds().processReceipt(pp_id_1_receipt, errors=STRICT)

()

A helper function to get the details of the investment. 

In [29]:
def get_invest_receipt(pp_id_1_receipt):
    events = sc_contract.events.PlayerNeedsFunds().processReceipt(pp_id_1_receipt)
    if len(events) > 0:
        return events[0]['args']
    else:
        return None

This will log the details of an investor event

In [30]:
invest_event1 = get_invest_receipt(invest_receipt)
print(invest_event1)
if(invest_event1):
    pass

AttributeDict({'amountReq': 10, 'play_id': 2})


In [31]:
exec_call(sc_contract, "player_database", pp_id_1)

[0,
 10,
 1000,
 True,
 False,
 202021,
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 10,
 500000,
 1000000,
 20]

Backer sends funds to player

In [32]:
_, send_funds_receipt = exec_transact_receipt(sc_contract, "backerTransPlayer", pp_id_1, 10, transaction={'from': pp_bank1})

At this point the player has receieved the funds for the tournament and the player will play the event.  Until the tournament is finished nothing else will happen.  There will be an Event listener waiting for tournament results from ResultOracle.  This will happen if the player receieves any money from the tournament, even if it's not a profit because the money to play was already sent.   

Compile ResultOracle.sol after tournament so that if necessary the player can distribute funds back to the backers.

In [33]:
# compile TemperatureOracle contract
result_oracle_compiled = compile_contract("ResultOracle")
# deploy TemperatureOracle contract
result_oracle_receipt, result_oracle_contract = deploy_contract(sc_w3, result_oracle_compiled, staking_chain_account)

In [34]:
result_oracle_contract.all_functions()

[<Function owner()>, <Function updateResults(uint256,uint256,uint256)>]

In [35]:
buyin_event1['tourney_id']

202021

Record results of tournament, for now the oracle is the contract owner but in the future the results will be verified by multiple sources.  

In [36]:
_, results_update_receipt_1 = exec_transact_receipt(result_oracle_contract, "updateResults", 10, pp_id_1, buyin_event1['tourney_id'])

In [37]:
result_oracle_contract.events.ResultUpdate().processReceipt(results_update_receipt_1)

(AttributeDict({'args': AttributeDict({'profit': 10,
   'pp_id': 0,
   'tourney_id': 202021}),
  'event': 'ResultUpdate',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd50e1d87cc34d65f3d50a5c27c8b12a2608473d843b241eb6375f65a4dc0f378'),
  'address': '0x51a240271AB8AB9f9a21C82d9a85396b704E164d',
  'blockHash': HexBytes('0x637baede3cac067a600f32d4120fe064315c790aaa94c8a959e8088523717564'),
  'blockNumber': 9}),)

The event listener that will pick up the results of the tournament so that funds can be distributed back to the backers.  

In [38]:
def get_results_event(results_update_receipt):
     return result_oracle_contract.events.ResultUpdate().processReceipt(results_update_receipt)[0]['args']

In [39]:
result_event_1 = get_results_event(results_update_receipt_1)
result_event_1

AttributeDict({'profit': 10, 'pp_id': 0, 'tourney_id': 202021})

If the player receieved any money from the tournament the funds are distributed back to the backer according to their action_owned

Backers listen for events for their profit

In [40]:
result_oracle_contract.events.ResultUpdate().processReceipt(results_update_receipt_1)

(AttributeDict({'args': AttributeDict({'profit': 10,
   'pp_id': 0,
   'tourney_id': 202021}),
  'event': 'ResultUpdate',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xd50e1d87cc34d65f3d50a5c27c8b12a2608473d843b241eb6375f65a4dc0f378'),
  'address': '0x51a240271AB8AB9f9a21C82d9a85396b704E164d',
  'blockHash': HexBytes('0x637baede3cac067a600f32d4120fe064315c790aaa94c8a959e8088523717564'),
  'blockNumber': 9}),)

The revenue from tourney is emitted as an event.  It's not the profit because the initial buy-in was already paid

In [41]:
_, results_receipt = exec_transact_receipt(sc_contract, "_updateResults", 10, pp_id_1, 202021, transaction={'from': backer_account1})

Getter function to see how much player needs to distribute to backers

In [42]:
revenue_event1 = exec_call(sc_contract, "get_amount_owed", pp_id_1, buyin_event1['tourney_id'])
revenue_event1

5

Distribute funds

In [43]:
_, backer_accept_profits_receipt_1 = exec_transact_receipt(sc_contract, "accept_results", pp_id_1, buyin_event1['tourney_id'], transaction={'from': pp_bank1,'value':revenue_event1})

Event listener for backers to ask for profits

In [44]:
sc_contract.events.pickUpProfit().processReceipt(backer_accept_profits_receipt_1)

(AttributeDict({'args': AttributeDict({'profit': 5,
   'tourney_id': 202021,
   'pp_id': 0}),
  'event': 'pickUpProfit',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xf268fb9fe9acf680bae9e39e32a7294c635b529b812b0bdd097494e58c2615a1'),
  'address': '0xF2E246BB76DF876Cef8b38ae84130F4F55De395b',
  'blockHash': HexBytes('0xb06ca5d63d2505d69f3e9d201c08cb48ba04f0a6c5d47cd8b555995ed208b04e'),
  'blockNumber': 11}),)

In [45]:
exec_call(sc_contract, 'action_owned', pp_id_1, backer_account1, buyin_event1['tourney_id'])

500000

In [46]:
exec_transact(sc_contract, "distribute_result", pp_id_1, buyin_event1['tourney_id'], transaction={'from':backer_account1})

'Profit sent, Balance cleared'

Make sure DApp properly clears action_owned fields after Balance has been cleared

In [47]:
exec_call(sc_contract, 'action_owned', pp_id_1, backer_account1, buyin_event1['tourney_id'])

0

In [48]:
# Test that backers can't withdraw their money twice
try:
    exec_transact(sc_contract, "distribute_result", pp_id_1, buyin_event1['tourney_id'], transaction={'from':backer_account1})
    exec_transact(sc_contract, "distribute_result", pp_id_1, buyin_event1['tourney_id'], transaction={'from':backer_account1})
    print("Test failed")
except Exception as e:
    print("Success:")
    print(e)

Success:
execution reverted: You don't own any of that player's action!


In [49]:
exec_call(sc_contract, "player_database", pp_id_1)

[0,
 0,
 1000,
 True,
 False,
 0,
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 0,
 0,
 0,
 0]

Test that multiple investors can invest in one tournament.  Many print statements to check state at various points.  

In [50]:
_, buyin_receipt3 = exec_transact_receipt(sc_contract, "request_buyin", pp_id_1, 25, 50, 102021, 2000000, transaction={'from': pp_bank1})
buyin_event3 = get_buyin_requested(buyin_receipt3)
print("buy-in requested:")
print(buyin_receipt3)
_, invest_receipt3 = exec_transact_receipt(sc_contract, "invest", 10, pp_id_1, transaction={'from': backer_account1, 'value': 10})
print("backer 1 invested")
_, invest_receipt4 = exec_transact_receipt(sc_contract, "invest", 10, pp_id_1, transaction={'from': backer_account2, 'value': 40})
print("backer 2 invested:")
invest_event4 = get_invest_receipt(invest_receipt4)
print(invest_event4)
_, results_update_receipt_2 = exec_transact_receipt(result_oracle_contract, "updateResults", 100, pp_id_1, buyin_event3['tourney_id'])
result_event_2 = get_results_event(results_update_receipt_2)
print("results received:")
print(result_event_2)
_, results_receipt = exec_transact_receipt(sc_contract, "_updateResults", 100, pp_id_1, 102021)
value = exec_call(sc_contract, "get_amount_owed", pp_id_1, buyin_event3['tourney_id'])
print("value = " + str(value))
_, backer_accept_profits_receipt_2 = exec_transact_receipt(sc_contract, "accept_results", pp_id_1, buyin_event3['tourney_id'], transaction={'from': pp_bank1,'value':value})
print("money received from backer")
exec_transact(sc_contract, "distribute_result", pp_id_1, buyin_event3['tourney_id'], transaction={'from':backer_account1})
print("backer 1 paid")
exec_transact(sc_contract, "distribute_result", pp_id_1, buyin_event3['tourney_id'], transaction={'from':backer_account2})
print("backer 2 paid")


buy-in requested:
AttributeDict({'transactionHash': HexBytes('0x33e14f0e8f4763489dfabe22ea7e89115e8aa6caec5a34d242f18dc8552dbd0d'), 'transactionIndex': 0, 'blockNumber': 13, 'blockHash': HexBytes('0x39837872e5f20c57d97b732c8ef078fb9bb60bda3285a5bebd0ffb0b69a3d6bc'), 'cumulativeGasUsed': 108960, 'gasUsed': 108960, 'contractAddress': None, 'logs': [AttributeDict({'type': 'mined', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x33e14f0e8f4763489dfabe22ea7e89115e8aa6caec5a34d242f18dc8552dbd0d'), 'blockHash': HexBytes('0x39837872e5f20c57d97b732c8ef078fb9bb60bda3285a5bebd0ffb0b69a3d6bc'), 'blockNumber': 13, 'address': '0xF2E246BB76DF876Cef8b38ae84130F4F55De395b', 'data': '0x0000000000000000000000000000000000000000000000000000000000000019000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000320000000000000000000000000000000000000000000000000000000000018e850000000000000000000000000000000000000000000000